In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('./output/test_hypothesis_evidences.csv')
test_df

,hypothesis,evidences,label
0,Women suffering from both claustrophobia and I...,"[""Exclusion Criteria:"", "" History of bilatera...",Contradiction
1,There are no conditions on mental mental healt...,"[""Inclusion Criteria:"", "" Women 18 years"", "" ...",Entailment
2,the secondary trial requires patients to be ov...,"[""Inclusion Criteria:"", "" Patients presenting...",Entailment
3,Patients currently taking part in the secondar...,"["" Participation in another clinical trial wi...",Entailment
4,HER2 + Patients in the primary trial receive t...,"[""INTERVENTION 1: "", "" Trastuzumab"", "" Parti...",Contradiction
...,...,...,...
495,alcoholic patients are excluded from the prima...,"["" Be non-smokers (no nicotine use within the...",Contradiction
496,Patients with a histologically/cytologically c...,"["" Non-small cell lung cancer"", "" Small cell...",Entailment
497,the primary trial treats one of its patient co...,"[""INTERVENTION 1: "", "" Pregabalin150"", "" Pat...",Entailment
498,after a month of TAK-228 Plus Tamoxifen treatm...,"[""Outcome Measurement: "", "" Ki67 Expression"",...",Contradiction


In [3]:
hypothesis_lst=test_df['hypothesis'].values.tolist()
len(hypothesis_lst)

500

In [4]:
evidence_lst=test_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

500

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

500

In [6]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, MegatronBertForSequenceClassification

In [7]:
# text_tok=AutoTokenizer.from_pretrained('bert-base-uncased')
# text_clf=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
#roberta-base
# text_tok=AutoTokenizer.from_pretrained('roberta-base')
# text_clf=AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=2)
#bio-bert
# text_tok=AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
# text_clf=AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1',num_labels=2)
#biomegatron
text_tok=BertTokenizer.from_pretrained('../transformer_models/biomegatron345uncased')
text_clf=MegatronBertForSequenceClassification.from_pretrained('../transformer_models/biomegatron345uncased',num_labels=2)

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at ../transformer_models/biomegatron345uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
class InputSequence:
    
    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True):
        
        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok(l_text,l_text2,padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label=np.array(l_label)
        print('tokenize done')
        
        self.batch_size=batch_size
        self.gpu=gpu
        
    def on_epoch_end(self):
        random.shuffle(self.data_idx)
        
    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]
        
        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=torch.from_numpy(
            self.l_label[batch_idx].astype(np.int64)
        )
        
        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=return_labels.cuda()
        
        return return_texts,return_labels
    
    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)
    

In [9]:
testing_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,batch_size=16,gpu=True)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

tokenize done


In [10]:
scores=[]
# model_names=['bert-base-uncased']+[
#     './output/clf_models/bert-base-uncased_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#roberta-base
# model_names=['roberta-base']+[
#     './output/clf_models/roberta-base_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#biobert
# model_names=['dmis-lab/biobert-v1.1']+[
#     './output/clf_models/biobert-v1.1_epoch_{}.pt'.format(format(epoch,'05d'))
#     for epoch in range(10)
# ]
#biomegatron
model_names=['../transformer_models/biomegatron345uncased']+[
    './output/clf_models/biomegatron345uncased_epoch_{}.pt'.format(format(epoch,'05d'))
    for epoch in range(10)
]
for model_name in model_names:
    scores.append([])
    clf=MegatronBertForSequenceClassification.from_pretrained(model_name).cuda()
    with torch.no_grad():
        for batch in range(len(testing_data)):
            batch_texts,batch_labels=testing_data[batch]
            scores[-1].append(F.softmax(clf(**batch_texts).logits,dim=1).detach().cpu().numpy())
            print('model:',model_name,'batch:',batch,end='\r')
    scores[-1]=np.concatenate(scores[-1],axis=0)
    clf.cpu()

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at ../transformer_models/biomegatron345uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for epoch in range(len(scores)):
    y_prob=scores[epoch][:,1]
    y_pred=[1 if a>0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch==0 else epoch,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['epoch','AVG_PREC','F1','PREC','REC','ACC'])

,epoch,AVG_PREC,F1,PREC,REC,ACC
0,pretrained,0.519633,0.211838,0.478873,0.136,0.494
1,1,0.539815,0.298246,0.554348,0.204,0.520
2,2,0.594687,0.413613,0.598485,0.316,0.552
3,3,0.587879,0.601036,0.528875,0.696,0.538
4,4,0.600983,0.480769,0.602410,0.400,0.568
5,5,0.600179,0.568507,0.581590,0.556,0.578
6,6,0.622298,0.580645,0.585366,0.576,0.584
7,7,0.635232,0.454308,0.654135,0.348,0.582
8,8,0.612131,0.570842,0.586498,0.556,0.582
9,9,0.629284,0.553097,0.618812,0.500,0.596
